# Evaluate Ensemble

This notebook combines the classification results of some models via logit-ensembling way.

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%cd ..
%load_ext autoreload
%autoreload 2

C:\Users\Minjae\Desktop\EEG_Project


In [2]:
# Load some packages
import os
import sys
import pickle
from copy import deepcopy
import hydra
from omegaconf import OmegaConf
from collections import OrderedDict

import numpy as np
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

import pprint
import wandb
import matplotlib
import matplotlib.pyplot as plt

# custom package
from datasets.caueeg_script import build_dataset_for_train
import models
from train.evaluate import check_accuracy
from train.evaluate import check_accuracy_extended
from train.evaluate import check_accuracy_extended_debug
from train.evaluate import check_accuracy_multicrop
from train.evaluate import check_accuracy_multicrop_extended
from train.evaluate import calculate_confusion_matrix
from train.evaluate import calculate_class_wise_metrics
from train.visualize import draw_roc_curve
from train.visualize import draw_confusion
from train.visualize import draw_class_wise_metrics
from train.visualize import draw_error_table
from train.visualize import annotate_heatmap

In [3]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 1.12.1+cu113
cuda is available.


-----

## List up the models to check accuracy

In [4]:
# model_names = [
#     'lo88puq7',
#     '2s1700lg',
#     'v301o425',
#     '1sl7ipca',
#     'gvqyvmrj',
#     'gjkysllw',
#     'xci5svkl',
#     '1vc80n1f',
#     'syrx7bmk',
# ]

model_names = [
    'tp7qn5hd',
    'q1hhkmik',
    '0svudowu',
    'ruqd8r7g',
    'dn10a6bv',
    'atbhqdgg',
    '4439k9pg',
    'nemy8ikm',
    '1cdws3t5',
]

model_pool = []

for model_name in model_names:
    path = os.path.join(r'E:\CAUEEG\checkpoint', model_name, 'checkpoint.pt')
    try:
        ckpt = torch.load(path, map_location=device)
        model_pool.append({'name': model_name, 'path': path})
    except Exception as e:
        print(e)
        print(f'- checkpoint cannot be opened: {path}')
        
pprint.pprint([model_dict['name'] for model_dict in model_pool])

['tp7qn5hd',
 'q1hhkmik',
 '0svudowu',
 'ruqd8r7g',
 'dn10a6bv',
 'atbhqdgg',
 '4439k9pg',
 'nemy8ikm',
 '1cdws3t5']


-----

## Redraw the metrics

In [5]:
task = 'exp-abnormal'
config = ckpt['config']
save_fig = True

# ensemble_multi_test_confusion = np.array([[5577, 162, 11], [1410, 3028, 687], [97, 1371, 2407]])
ensemble_multi_test_confusion = np.array([[3262, 2442], [1072, 10088]])

output_path = f"local/output/imgs/ensemble_test/"
os.makedirs(output_path, exist_ok=True)

ensembel_multi_test_class_wise_metrics = calculate_class_wise_metrics(ensemble_multi_test_confusion)     

save_path = os.path.join(output_path, f'{task}-ensemble-confusion-tta.pdf') if save_fig else None
draw_confusion(ensemble_multi_test_confusion, config['class_label_to_name'], 
               normalize=True, use_wandb=False, save_path=save_path)

save_path = os.path.join(output_path, f'{task}-ensemble-class-wise-tta.pdf') if save_fig else None
draw_class_wise_metrics(ensemble_multi_test_confusion, config['class_label_to_name'], 
                        use_wandb=False, save_path=save_path)